# Практическая работа №4: Элементы корреляционного анализа. Проверка статистической гипотезы о равенстве коэффициента корреляции нулю

Выполнил студент гр. 1384 Шаганов Вячеслав.

## Цель работы

Освоение основных понятий, связанных с корреляционной зависимостью между случайными величинами, статистическими гипотезами и проверкой их «справедливости».


## Основные теоретические положения
В данной работе рассматриваются методы корреляционного анализа и проверка статистических гипотез. Основные теоретические аспекты:

1. Корреляционная зависимость

    Корреляционная зависимость между двумя случайными величинами $ X $ и $ Y $ характеризуется коэффициентом корреляции Пирсона:
    $$
    r_{xy} = \frac{\text{cov}(X, Y)}{\sigma_X \sigma_Y}
    $$,
    где $ \text{cov}(X, Y) $ — ковариация, $ \sigma_X $ и $ \sigma_Y $ — стандартные отклонения $ X $ и $ Y $.

2. Доверительный интервал для коэффициента корреляции

    Для построения доверительного интервала используется преобразование Фишера:
    $$
    z = \frac{1}{2} \ln\left(\frac{1 + r_{xy}}{1 - r_{xy}}\right)
    $$.
    Доверительный интервал для $ z $:
    $$
    z \pm z_{\gamma} \cdot \frac{1}{\sqrt{n - 3}}
    $$,
    где $ z_{\gamma} $ — квантиль стандартного нормального распределения для уровня значимости $ \gamma $, $ n $ — объём выборки.

3. Проверка гипотезы о равенстве коэффициента корреляции нулю

    Гипотеза $ H_0: r_{xy} = 0 $ проверяется с использованием статистики:
    $$
    t = \frac{r_{xy} \sqrt{n - 2}}{\sqrt{1 - r_{xy}^2}}
    $$.
    Если $ |t| > t_{\text{крит}} $, гипотеза отвергается, иначе принимается.

4. Корреляционная таблица

    Корреляционная таблица представляет собой двумерную таблицу частот, где строки соответствуют интервалам значений $ X $, а столбцы — интервалам значений $ Y $.

Эти методы позволяют анализировать взаимосвязь между случайными величинами и проверять статистические гипотезы.

## Постановка задачи

Из заданной генеральной совокупности сформировать выборку по второму признаку. Провести статистическую обработку второй выборки в объёме практических работ №1 и №2, с целью определения точечных статистических оценок параметров распределения исследуемого признака (математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса и коэффициента вариации). Для системы двух случайных величин $ X $ (первый признак) и $ Y $ (второй признак) сформировать двумерную выборку и найти статистическую оценку коэффициента корреляции, построить доверительный интервал для коэффициента корреляции и осуществить проверку статистической гипотезы о равенстве коэффициента корреляции нулю. Полученные результаты содержательно проинтерпретировать.

## Выполнение работы

#### Задание 1. Провести статистическую обработку второй выборки в объёме практических работ №1 и №2, с целью определения точечных статистических оценок параметров распределения исследуемого признака (математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса, моды, медианы и коэффициента вариации). Оформить результаты в виде таблицы, сделать выводы.

Так как в П.Р. №1 и №2 данная обработка уже представлена, просто перенесём результаты в данную работу 

In [230]:
import numpy as np

with open('sample.txt', 'r') as f:
    sample = [[float(e) for e in s.split()] for s in f.readlines()]
print("Размер выборки:", len(sample))

sample = np.array(sample)
X, Y = sample[:, 0], sample[:, 1]

class SampleData:
    def __init__(self, series):
        if isinstance(series, SampleData):
            series = series.series
        self.series = series
        ranked = np.sort(series)
        var = np.array( [ranked[0]] + [ranked[i] for i in range(1, len(ranked)) if ranked[i] != ranked[i-1]] )
        k = 1 + 3.31 * np.log10(len(series))
        self.k = int(k) if int(k) % 2 == 1 else int(k) + 1
        k = self.k

        minval, maxval = var[[0, -1]]
        h = (maxval - minval) / k
        self.h = h

        self.intervals = [[minval + i*h, minval + (i+1)*h] for i in range(k)]

        self.bins = [np.average(b) for b in self.intervals] # Середины интервалов 
        self.abs_counts = np.array([
            len(   
                series[    # получаем элементы по фильтру
                    (series < self.intervals[i][1])   # отбираем элементы, меньшие x_{i+1}, где [x_i, x_{i+1}) - интервал
                    | ((i == k-1) & (series <= self.intervals[i][1]))   # учитываем, что последний интервал имеет нестрогую правую границу
                ]
            ) for i in range(k)
        ])

        self.rel_counts = self.abs_counts / len(series)

        self.abs_freq = [self.abs_counts[i] - (0 if i == 0 else self.abs_counts[i-1]) for i in range(k)]
        self.rel_freq = [self.rel_counts[i] - (0 if i == 0 else self.rel_counts[i-1]) for i in range(k)]

        self.C = self.bins[len(self.bins) // 2]
        self.u = (self.bins - self.C) / h

        self.u_series = np.array([0]*120)
        for i, x in enumerate(series):
            for interval_i, interval in enumerate(self.intervals):
                if x < interval[1] or interval_i == k-1:
                    self.u_series[i] = interval_i - k // 2
                    break 

        nu = [self.rel_freq * self.u**k for k in range(5)]
        Ms = [ sum(nu[i]) for i in range(len(nu)) ]
        m  = [
            1, 
            0,
            (Ms[2] - Ms[1]**2)*h**2,
            (Ms[3] - 3*Ms[2]*Ms[1] + 2*Ms[1]**3)*h**3,
            (Ms[4] - 4*Ms[3]*Ms[1] + 6*Ms[2]*Ms[1]**2 - 3*Ms[1]**4)*h**4
        ]

        self.interval_mean = np.average(self.bins, weights=self.rel_freq)
        self.interval_disp = np.average((self.bins - self.interval_mean) ** 2, weights=self.rel_freq)
        
        self.u_mean = Ms[1] * h + self.C
        self.u_disp = m[2]

        s2 = len(series) / (len(series)-1) * self.u_disp
        self.s = np.sqrt(s2)

        self.As = m[3] / self.s**3
        self.E = m[4] / self.s**4 - 3

        # Mode 
        i = np.argmax(self.abs_freq)
        self.Mo = self.bins[i-1] + h * (self.abs_freq[i] - self.abs_freq[i-1]) / ((self.abs_freq[i] - self.abs_freq[i-1]) + (self.abs_freq[i] - self.abs_freq[i+1]))

        # Median
        i = 0
        for j in range(len(self.abs_counts)):
            if self.rel_counts[j] >= 0.5:
                i = j
                break
        self.Me = self.bins[i-1] + h / self.abs_freq[i] * (0.5 - sum([h/self.abs_freq[j] for j in range(i)]))

        # Var coef
        self.mean = np.mean(series)
        self.V = self.s / self.mean
    
X = SampleData(X)
Y = SampleData(Y)

Размер выборки: 120


In [231]:
# Код для вывода таблицы
'''
print("Sample", "Mean", "Dispersion", "s", "As", "E", "Mo", "Me", "V", sep=' | ')
for S in [X, Y]:
    print("", *[round(v, 4) for v in [S.u_mean, S.u_disp, S.s, S.As, S.E, S.Mo, S.Me, 100*S.V]], sep=' | ')''';

| Sample | Mean | Dispersion | s | As | E | Mo | Me | V |
|---|---|---|---|---|---|---|---|---|
| X | 449.8929 | 2975.4069 | 54.776 | 0.1207 | -0.3342 | 429.4821 | 406.0685 | 12.1753 |
| Y | 129.3805 | 484.0076 | 22.0924 | -0.2918 | 0.1342 | 118.4804 | 115.9574 | 17.0755 |

#### Задание 2. Построить двумерный интервальный вариационный ряд, оформить в виде таблицы.

In [232]:
table = [[0]*X.k for _ in range(Y.k)]
for i in range(len(X.u_series)):
    table[X.u_series[i]+X.k//2][Y.u_series[i]+Y.k//2] += 1



# Код для вывода таблицы
def rnd(a, d=4):
    return 0 if a < 10**-d else round(a, d)
def rndlist(l, d=4):
    return [rnd(a, d) for a in l]
'''
print('| Y \\ X', *rndlist(X.bins), '', sep=' | ')
print()
for i in range(X.k):
    print('', rnd(Y.bins[i]), *table[i],  Y.abs_freq[i], '', sep=' | ')
''';

| Y \ X | 340.2857 | 378.8571 | 417.4286 | 456.0 | 494.5714 | 533.1429 | 571.7143 |
|----|----|----|----|----|----|----|----|
 | 70.8786 | 3 | 2 | 1 | 0 | 0 | 0 | 0 | 3 | 
 | 87.4357 | 0 | 4 | 6 | 4 | 0 | 0 | 0 | 6 | 
 | 103.9929 | 0 | 0 | 4 | 20 | 5 | 0 | 0 | 11 | 
 | 120.55 | 0 | 0 | 0 | 14 | 19 | 1 | 0 | 39 | 
 | 137.1071 | 0 | 0 | 0 | 1 | 9 | 11 | 2 | 35 | 
 | 153.6643 | 0 | 0 | 0 | 0 | 2 | 3 | 4 | 17 | 
 | 170.2214 | 0 | 0 | 0 | 0 | 0 | 2 | 3 | 9 | 

#### Задание 3. По полученному двумерному интервальному вариационному ряду построить корреляционную таблицу, сделать выводы.

In [ ]:
X_mults = [[0]*X.k for _ in range(Y.k)]
Y_mults = [[0]*Y.k for _ in range(X.k)]
for i in range(Y.k):
    for j in range(X.k):
        X_mults[i][j] = X.bins[j] * table[i][j]
        Y_mults[i][j] = Y.bins[i] * table[i][j]

Y_mults_sums = [sum([row[i] for row in Y_mults]) for i in range(Y.k)] # column sum
X_mults_sums = [sum(row) for row in X_mults] # row sum

yiXi = [Y.bins[i] * X_mults_sums[i] for i in range(Y.k)]
xjYj = [X.bins[j] * Y_mults_sums[j] for j in range(X.k)]
# Код для вывода таблицы
'''
print('| Y \\ X', *rndlist(X.bins), '$X_i=\\sum{n_{ij}x_j}$', '$y_i X_i$', '', sep=' | ')
print("|----"*(X.k+3)+"|")
for i in range(X.k):
    print(
        '', rnd(Y.bins[i]), 
        *[f"{rnd(y, 2)} \\ {rnd(x, 2)}" for y,x in zip(Y_mults[i], X_mults[i])], 
        rnd(X_mults_sums[i]), 
        rnd(yiXi[i], 2), '', 
        sep=' | ')
print('| $Y_j=\\sum{n_{ij}y_i}$', *rndlist(Y_mults_sums), '', f"$\\sum{{y_i X_i}} = {rnd(sum(yiXi))}$ | ", sep=' | ')
print('| $x_j Y_j$', *rndlist(xjYj, 2), f"$\\sum{{x_j Y_j}} = {rnd(sum(xjYj))}$ | ", '', sep=' | ')
''';

| Y \ X | 340.2857 | 378.8571 | 417.4286 | 456.0 | 494.5714 | 533.1429 | 571.7143 | $X_i=\sum{n_{ij}x_j}$ | $y_i X_i$ | 
|----|----|----|----|----|----|----|----|----|----|
 | 70.8786 | 212.64 \ 1020.86 | 141.76 \ 757.71 | 70.88 \ 417.43 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 2196.0 | 155649.34 | 
 | 87.4357 | 0 \ 0 | 349.74 \ 1515.43 | 524.61 \ 2504.57 | 349.74 \ 1824.0 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 5844.0 | 510974.31 | 
 | 103.9929 | 0 \ 0 | 0 \ 0 | 415.97 \ 1669.71 | 2079.86 \ 9120.0 | 519.96 \ 2472.86 | 0 \ 0 | 0 \ 0 | 13262.5714 | 1379212.7 | 
 | 120.55 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 1687.7 \ 6384.0 | 2290.45 \ 9396.86 | 120.55 \ 533.14 | 0 \ 0 | 16314.0 | 1966652.7 | 
 | 137.1071 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 137.11 \ 456.0 | 1233.96 \ 4451.14 | 1508.18 \ 5864.57 | 274.21 \ 1143.43 | 11915.1429 | 1633651.19 | 
 | 153.6643 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 307.33 \ 989.14 | 460.99 \ 1599.43 | 614.66 \ 2286.86 | 4875.4286 | 749179.25 | 
 | 170.2214 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 0 \ 0 | 340.44 \ 1066.29 | 510.66 \ 1715.14 | 2781.4286 | 473458.74 | 
| $Y_j=\sum{n_{ij}y_i}$ | 212.6357 | 491.5 | 1011.4643 | 4254.4071 | 4351.7071 | 2430.1643 | 1399.5357 |  | $\sum{y_i X_i} = 6868778.2408$ |
| $x_j Y_j$ | 72356.9 | 186208.29 | 422214.09 | 1940009.66 | 2152230.02 | 1295624.73 | 800134.56 | $\sum{x_j Y_j} = 6868778.2408$ |  |

Можно видеть, что значения $\sum{x_j Y_j}$ и $\sum{y_i X_i}$ совпали, что говорит о корректности вычислений.

#### Задание 4. Исходя из результатов корреляционной таблицы вычислить значение выборочного коэффициента корреляции двумя способами: с помощью стандартной формулы и с помощью условных вариант. Убедиться, что результаты совпадают. Сделать выводы.

$$
\bar{r_{xy}} = \frac{\sum_{i=1}^{K_y}\sum_{j=1}^{K_x}{n_{ij} y_i x_j} - N\bar{y}\bar{x}}{N s_x s_y} = \frac{\sum\sum{n_{ij} u_i v_j} - N\bar{u}\bar{v}}{N \bar{\sigma_u} \bar{\sigma_v}}
$$
причём 
$$
\sum_{i=1}^{K_y}\sum_{j=1}^{K_x}{n_{ij} y_i x_j} = \sum_{i=1}^{K_y}{y_i \sum_{j=1}^{K_x}{x_j n_{ij}}} = \sum_{j=1}^{K_x}{x_j \sum_{i=1}^{K_y}{y_i n_{ij}}} = \sum_{i=1}^{K_y}{y_i X_i} = \sum_{j=1}^{K_x}{x_j Y_j}

In [443]:
sigma = lambda S: np.sqrt( (S.series**2).mean() - S.series.mean()**2 )
Xbinseries = np.array(X.bins)[X.u_series + X.k//2]
Ybinseries = np.array(Y.bins)[Y.u_series + Y.k//2]
r_xy = ( np.sum(Xbinseries * Ybinseries) - len(Y.series) * X.interval_mean * Y.interval_mean ) / ( len(Y.series) * np.sqrt(X.interval_disp * Y.interval_disp) )
sum_ujVj = 0

for i in range(Y.k):
    for j in range(X.k):
        sum_ujVj += table[i][j] * (X.u[j] * Y.u[i])

sigma_u = lambda S: np.sqrt( (S.u_series**2).mean() - S.u_series.mean()**2 )

r_uv = ( sum_ujVj - len(Y.u_series) * X.u_series.mean() * Y.u_series.mean() ) / \
        ( len(Y.u_series) * sigma_u(X) * sigma_u(Y) )

r_xy, r_uv

(np.float64(0.8653704587990387), np.float64(0.8653704587990426))

Как можно видеть, результаты совпали

#### Задание 5. Построить доверительный интервал для коэффициента корреляции при уровне значимости $ \gamma \in \{0.95, 0.99\} $, сделать выводы.

In [447]:
import scipy.stats as stats

z = 1/2 * np.log( (1 + r_xy) / (1 - r_xy) )
sigma_z = 1 / np.sqrt(len(X.series) - 3)

In [448]:
gamma = 0.95
lambda_gamma = stats.norm.ppf(gamma / 2)
left_z, right_z = sorted([z-lambda_gamma*sigma_z, z+lambda_gamma*sigma_z])
print(f"Доверительный интервал для генерального значения уровня доверия {gamma}: [{left_z:.4f}, {right_z:.4f}]")
z_inv = lambda z: (np.exp(2*z) - 1)/(np.exp(2*z) + 1)
print(f"Доверительный интервал для коэффициента корреляции уровня доверия {gamma}: [{z_inv(left_z):.5f}, {z_inv(right_z):.5f}]")

Доверительный интервал для генерального значения уровня доверия 0.95: [1.3085, 1.3201]
Доверительный интервал для коэффициента корреляции уровня доверия 0.95: [0.86391, 0.86682]


In [449]:
gamma = 0.99
lambda_gamma = stats.norm.ppf(gamma / 2)
left_z, right_z = sorted([z-lambda_gamma*sigma_z, z+lambda_gamma*sigma_z])
print(f"Доверительный интервал для генерального значения уровня доверия {gamma}: [{left_z:.4f}, {right_z:.4f}]")
z_inv = lambda z: (np.exp(2*z) - 1)/(np.exp(2*z) + 1)
print(f"Доверительный интервал для коэффициента корреляции уровня доверия {gamma}: [{z_inv(left_z):.5f}, {z_inv(right_z):.5f}]")

Доверительный интервал для генерального значения уровня доверия 0.99: [1.3132, 1.3155]
Доверительный интервал для коэффициента корреляции уровня доверия 0.99: [0.86508, 0.86566]


При увеличении значения уровня доверия границы доверительных интервалов сужаются, что ожидаемо

#### Задание 6. Осуществить проверку статистической гипотезы о равенстве коэффициента корреляции нулю при заданном уровне значимости $ \alpha = 0.05 $, сделать выводы

In [462]:
alpha = 0.05
T = r_xy * np.sqrt(len(X.series) - 2) / np.sqrt(1 - r_xy**2)
T_crit = stats.t.ppf(1-alpha, len(X.series) - 2)
print(f"Наблюдаеиое значение критерия T {T}")
print(f"Критическое значение критерия T {T_crit}")

Наблюдаеиое значение критерия T 18.75816445222528
Критическое значение критерия T 1.6578695221021444


Так как критическое значение критерия на порядок меньше наблюдаемого, основная гипотеза отличается -- коэффициент корреляции $\bar{r_{xy}}$ значимо отличается от нуля 

## Выводы

В ходе работы были проведены статистическая обработка выборки, вычислены точечные оценки параметров распределения (математическое ожидание, дисперсия, СКО и др.), а также построен двумерный интервальный вариационный ряд и корреляционная таблица. Выборочный коэффициент корреляции был рассчитан двумя способами, результаты совпали. Построен доверительный интервал для коэффициента корреляции при уровнях значимости 0.95 и 0.99. Проверка гипотезы о равенстве коэффициента корреляции нулю при уровне значимости 0.05 показала, что гипотеза отвергается. Полученные результаты подтверждают наличие корреляционной зависимости между исследуемыми признаками.